# Benchmark

This notebook provides a straightforward way to compare the PyZX optimization routines against other approaches on a standard set of benchmark circuits.

First we execute the standard set of imports:

In [1]:
import random, math, os, time
import sys; sys.path.append('..')
import pyzx as zx
%config InlineBackend.figure_format = 'svg'

The following class is some boilerplate around the simplification routines so that we can more easily print the relevant metrics

In [2]:
class CircuitComparer:
    def __init__(self, dirname, before, after):
        self.fname_before = os.path.join(dirname, before)
        if after:
            self.fname_after = os.path.join(dirname, after)
        else:
            self.fname_after = ""
        self.fname_tpar = ""
        if before.find('before') != -1:
            self.name = before[:-7]
        else:
            self.name = before
        self.has_run = False
    def __str__(self):
        return "CircuitComparer({}, {})".format(self.name, str(self.has_run))
    def __repr__(self):
        return str(self)
    
    def run(self):
        if self.has_run: return True
        if self.fname_after:
            c = zx.Circuit.from_quipper_file(self.fname_after).to_basic_gates()
            self.nrscm2 = c.twoqubitcount()
            self.nrscmtotal = len(c.gates)
        else:
            self.nrscm2 = '-'
            self.nrscmtotal = '-'
        if self.fname_tpar:
            c2 = zx.Circuit.load(self.fname_tpar).to_basic_gates()
            self.tpar2 = c2.twoqubitcount()
            self.tpartotal = len(c2.gates)
        else: 
            self.tpar2 = "-"
            self.tpartotal = '-'
        
        c = zx.Circuit.load(self.fname_before).to_basic_gates()
        self.qubits = c.qubits
        self.gatecount = len(c.gates)
        self.cnotcount = c.twoqubitcount()
        g = c.to_graph()
        t = time.time()
        g = zx.simplify.teleport_reduce(g)
        self.time_simpl = time.time() - t
        t = time.time()
        c = zx.Circuit.from_graph(g).split_phase_gates()
        c = zx.optimize.basic_optimization(c).to_basic_gates().split_phase_gates()
        self.circuit_opt = c
        self.pyzx2 = c.twoqubitcount()
        self.pyzxtotal = len(c.gates)
        self.time_opt = time.time() - t
        self.has_run = True
        return True
    
    def get_output(self):
        if not self.has_run:
            self.run()
        s = self.name.ljust(20) + str(self.qubits).rjust(7)
        s += " | " + str(self.gatecount).rjust(5) + str(self.cnotcount).rjust(6) + " | "
        s += str(self.nrscmtotal).rjust(6) + str(self.nrscm2).rjust(8) 
        s += " | " + str(self.tpartotal).rjust(6) + str(self.tpar2).rjust(6) + " | "
        s += str(self.pyzxtotal).rjust(6) + str(self.pyzx2).rjust(6)
        s += "{:.2f}".format(self.time_simpl).rjust(9)
        s += "{:.2f}".format(self.time_opt).rjust(10)
        return s

Next we define a function that loads in a directory of circuit files. Note that the directory we target has up to 3 versions of each circuit:

* circuit_before   - This is the original circuit with any modifications, taken from the [Github page](https://github.com/njross/optimizer) of [[1]](#NRSCM)
* circuit_after    - This is the circuit produced by the optimization routines of [[1]](#NRSCM).
* circuit_tpar.qc  - This is the circuit produced by the Tpar algorithm [[2]](#Tpar).
  
<a id="NRSCM"></a>
[1] [Nam, Ross, Su, Childs, Maslov - Automated optimization of large quantum circuits with continuous parameters](https://www.nature.com/articles/s41534-018-0072-4)

<a id="Tpar"></a>
[2] [Amy, Maslov, Mosca - Polynomial-time T-depth Optimization of Clifford+T circuits via Matroid Partitioning](https://arxiv.org/abs/1303.2042)

In [3]:
def load_circuits(directory):
    d = directory
    beforefiles = []
    afterfiles = []
    tparfiles = []
    for f in os.listdir(d):
        if not os.path.isfile(os.path.join(d,f)): continue
        if f.find('before') != -1:
            beforefiles.append((f,d))
        elif f.find('tpar') != -1:
            tparfiles.append((f,d))
        elif f.find('.qc') != -1 or f.find('.tfc') != -1:
            beforefiles.append((f,d))
        else: afterfiles.append((f,d))
    
    circuits = []
    for f, d in beforefiles:
        if f.find('before') == -1:
            n = os.path.splitext(f)[0]
        else: n = f[:-7]
        for f2,d2 in afterfiles:
            if d!=d2: continue
            if f2.startswith(n):
                c = CircuitComparer(d, f, f2)
                circuits.append(c)
                break
        else:
            c = CircuitComparer(d, f, '')
            circuits.append(c)
        for f2,d2 in tparfiles:
            if d!=d2: continue
            if f2.startswith(n):
                circuits[-1].fname_tpar = os.path.join(d2,f2)
    
    return circuits

dir_fast_circuits = os.path.join('..', 'circuits', 'Fast')
fast_circuits = load_circuits(dir_fast_circuits)

The directory we target contains a subset of all benchmark circuits, chosen for given quick results. The following cell giving benchmark results of these circuits should therefore only take a few seconds to run. For the benchmarks of slower circuits see [below](#slowbench).
The columns have the following meaning:

* `Circuit     ` - The name of the circuit
* `qubits      ` - Amount of qubits in the circuit
* `G-count     ` - Gate count of original circuit
* `2-count     ` - Amount of 2-qubit gates in original circuit
* `G/2-NRSCM   ` - Total amount and 2-qubit gate amount from optimised circuit of [[1]](#NRSCM)
* `G/2-Tpar    ` - Total amount and 2-qubit gate amount from optimised circuit of [[2]](#Tpar)
* `G/2-PyZX    ` - Total amount and 2-qubit gate amount from optimised circuit made by PyZX
* `Time-Simp   ` - The time taken for running the simplification routine on the circuit
* `Time-Extract` - The time taken for extracting the circuit after the simplification

Note that not all circuits were present in the papers [[1]](#NRSCM) and [[2]](#Tpar) in which case the relevant columns are empty.

In [4]:
print("Circuit".ljust(20), "qubits", "G-count", "2-count", "G-NRSCM", "2-NRSCM", " G-Tpar", "2-Tpar", "G-PyZX", "2-PyZX", "Time-Simp", "Time-Opt")
for c in fast_circuits:
    print(c.get_output())

Circuit              qubits G-count 2-count G-NRSCM 2-NRSCM  G-Tpar 2-Tpar G-PyZX 2-PyZX Time-Simp Time-Opt
Adder8                   23 |   637   243 |    190      94 |      -     - |    362   195     0.35      0.06
adder_8                  24 |  1014   409 |    606     291 |   1280   885 |    690   351     0.77      0.11
barenco_tof_10           19 |   514   192 |    264     130 |    517   328 |    365   176     0.25      0.06
barenco_tof_3             5 |    66    24 |     40      18 |     82    54 |     50    22     0.03      0.01
barenco_tof_4             7 |   130    48 |     72      34 |    141    90 |     95    44     0.04      0.01
barenco_tof_5             9 |   194    72 |    104      50 |    206   132 |    140    66     0.09      0.02
csla_mux_3_original      15 |   190    80 |    155      70 |      -     - |    156    75     0.08      0.03
csum_mux_9_corrected     30 |   448   168 |    266     140 |      -     - |    308   168     0.19      0.06
gf2^10_mult              30 

<a id="slowbench"></a>
And now we do the benchmark on the slower circuits. Note that this can take a while to complete

In [5]:
dir_slow_circuits = os.path.join('..', 'circuits', 'Slow')
slow_circuits = load_circuits(dir_slow_circuits)
print("Circuit".ljust(20), "qubits", "G-count", "2-count", "G-NRSCM", "2-NRSCM", " G-Tpar", "2-Tpar", "G-PyZX", "2-PyZX", "Time-Simp", "Time-Opt")
for c in slow_circuits:
    print(c.get_output())

Circuit              qubits G-count 2-count G-NRSCM 2-NRSCM  G-Tpar 2-Tpar G-PyZX 2-PyZX Time-Simp Time-Opt
Adder16                  47 |  1437   547 |    414     206 |      -     - |    786   435     0.79      0.15
Adder32                  95 |  3037  1155 |    862     430 |      -     - |   1618   883     2.04      0.37
Adder64                 191 |  6237  2371 |   1758     878 |      -     - |   3338  1835     5.43      1.24
gf2^16_mult              48 |  4397  1581 |   2707    1581 |   7714  6592 |   2686  1581     5.33      0.42
ham15-high.qc            20 |  6010  2149 |      -       - |      -     - |   3651  1795     9.12      0.95
ham15-med.qc             17 |  1436   534 |      -       - |      -     - |    869   430     1.50      0.19
mod_adder_1024           28 |  4855  1720 |   2736    1278 |   5183  3540 |   3035  1392     3.71      0.64
QFT32                    32 |  1562   612 |   1012     612 |      -     - |   1012   612     0.51      0.12
QFTAdd16                 32 